In [ ]:
import os
from dotenv import load_dotenv



True

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [3]:
llm = ChatOpenAI(model = 'gpt-4o', temperature=0)

prompt_extract = ChatPromptTemplate.from_template(
    "Extract the technical specifications from the following text:{text_input}"
)

prompt_transform = ChatPromptTemplate.from_template(
    "Transform the following specifications into a JSON object with'cpu', 'memory', and 'storage' as keys: {specifications}"
)

In [4]:
extraction_chain = prompt_extract | prompt_transform | StrOutputParser()

In [7]:
full_chain = (
    {"specification" : extraction_chain}
    | prompt_transform 
    | llm 
    | StrOutputParser()
)

In [12]:
input_text = "The new laptop model features a 3.5 GHz octa-core processor, 16GB of RAM, and a 1TB NVMe SSD."

In [11]:
final_result = full_chain.invoke({"text_input" : input_text})

ValidationError: 1 validation error for Generation
text
  Input should be a valid string [type=string_type, input_value=ChatPromptValue(messages=... response_metadata={})]), input_type=ChatPromptValue]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://eu.api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


In [14]:
# load_dotenv()
# Make sure your OPENAI_API_KEY is set in the .env file
# Initialize the Language Model (using ChatOpenAI is recommended)
llm = ChatOpenAI(temperature=0)
# --- Prompt 1: Extract Information ---
prompt_extract = ChatPromptTemplate.from_template(
"Extract the technical specifications from the followingtext:\n\n{text_input}"
)
# --- Prompt 2: Transform to JSON ---
prompt_transform = ChatPromptTemplate.from_template(
"Transform the following specifications into a JSON object with'cpu', 'memory', and 'storage' as keys:\n\n{specifications}"
)
# --- Build the Chain using LCEL ---
# The StrOutputParser() converts the LLM's message output to a simple string.
extraction_chain = prompt_extract | llm | StrOutputParser()
# The full chain passes the output of the extraction chain into the
'specifications'
# variable for the transformation prompt.
full_chain = (
{"specifications": extraction_chain}
| prompt_transform
| llm
| StrOutputParser()
)
# --- Run the Chain ---
input_text = "The new laptop model features a 3.5 GHz octa-coreprocessor, 16GB of RAM, and a 1TB NVMe SSD."
# Execute the chain with the input text dictionary.
final_result = full_chain.invoke({"text_input": input_text})
print("\n--- Final JSON Output ---")
print(final_result)

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://eu.api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://eu.api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')



--- Final JSON Output ---
{
    "cpu": "3.5 GHz octa-core",
    "memory": "16GB",
    "storage": "1TB NVMe SSD"
}


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://eu.api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


In [17]:
def booking_handler(request: str) -> str:
    """Simulates the booking agent handling a request"""
    print("\n ---- DELEGATING TO BOOKING HANDLER ---- \n")
    return f"Booking Handler processed request: '{request}'. Result: Simulated booking action."
def info_handler(request: str) -> str:
    """Simulates the Info Agent handling a request."""
    print("\n--- DELEGATING TO INFO HANDLER ---")
    return f"Info Handler processed request: '{request}'. Result:Simulated information retrieval."
def unclear_handler(request: str) -> str:
    """Handles requests that couldn't be delegated."""
    print("\n--- HANDLING UNCLEAR REQUEST ---")
    return f"Coordinator could not delegate request: '{request}'.Please clarify."

In [18]:
router_promt = ChatPromptTemplate.from_messages([
    "system", """
        Analyze the users request and determine which specialist hanlder should process it.
        - If the request is related to booking flights or hotels output 'booker'
        - For all the general information questions, output 'info'
        - If the request is unclear or doesnt fit either category, output unclear.
        
        Only output one word: 'booker', 'info', or 'unclear'.
    """,
    ("user", "{request}")
])

In [19]:
if llm:
    router_chain = router_promt | llm | StrOutputParser()

In [ ]:
if router_chain:
    print("Hello ")

Hello 


In [25]:
from langchain_core.runnables import RunnablePassthrough, RunnableBranch

branches = {
"booker": RunnablePassthrough.assign(output=lambda x:
booking_handler(x['request']['request'])),
"info": RunnablePassthrough.assign(output=lambda x:
info_handler(x['request']['request'])),
"unclear": RunnablePassthrough.assign(output=lambda x:
unclear_handler(x['request']['request'])),
}

In [28]:
delegation_branch = RunnableBranch(
    (lambda x : x['decision'].strip() == 'booker', branches['booker']),
    (lambda x : x['decision'].strip() == 'info', branches['info']),
    branches['unclear']
)

In [ ]:
coord_agent = {
    "decision" : router_chain,
    "request" : RunnablePassthrough()
} | delegation_branch | (lambda x: x['output'])

In [30]:
request_a = "Book me a flight to London."

In [31]:
result_a = coord_agent.invoke({"request": request_a})

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://eu.api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')



 ---- DELEGATING TO BOOKING HANDLER ---- 



Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://eu.api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


In [32]:
result_a

"Booking Handler processed request: 'Book me a flight to London.'. Result: Simulated booking action."

In [ ]:
request_c = "Tell me about quantum physics."

In [ ]:
result_c = coord_agent.invoke({"request": request_c})


--- DELEGATING TO INFO HANDLER ---


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://eu.api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://eu.api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


In [35]:
result_c

"Info Handler processed request: 'Tell me about quantum physics.'. Result:Simulated information retrieval."

In [36]:
request_d = "I don't know what i need - so what do i need again?"

In [ ]:
result_d = coord_agent.invoke({"request": request_d})


--- HANDLING UNCLEAR REQUEST ---


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://eu.api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://eu.api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


In [38]:
result_d

"Coordinator could not delegate request: 'I don't know what i need - so what do i need again?'.Please clarify."

In [40]:
coord_agent_full = {
    "decision" : router_chain,
    "request" : RunnablePassthrough()
} | delegation_branch 

In [41]:
coord_agent_full.invoke({"request": request_d})

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://eu.api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')



--- HANDLING UNCLEAR REQUEST ---


{'decision': 'unclear',
 'request': {'request': "I don't know what i need - so what do i need again?"},
 'output': "Coordinator could not delegate request: 'I don't know what i need - so what do i need again?'.Please clarify."}

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://eu.api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
